# Upload custom data

In [1]:
from datetime import datetime

import geopandas
import pandas as pd

import geoengine as ge

## Initialize Geo Engine

In [2]:
ge.initialize("http://localhost:3030/api")

## Define dataframe

In [3]:
df = pd.DataFrame({"label": ["NA", "DE"], "index": [0, 1], "rnd": [34.34, 567.547]})

polygons = [
    "Polygon((-121.46484375 47.109375, -99.31640625 17.2265625, -56.42578125 52.03125,-121.46484375 47.109375))",
    "Polygon((4.74609375 53.61328125, 5.09765625 43.06640625, 15.1171875 43.76953125, 15.1171875 54.4921875, 4.74609375 53.61328125))",  # noqa: E501
]

gdf = geopandas.GeoDataFrame(df, geometry=geopandas.GeoSeries.from_wkt(polygons), crs="EPSG:4326")

gdf

,label,index,rnd,geometry
0,NA,0,34.340,"POLYGON ((-121.46484 47.10938, -99.31641 17.22..."
1,DE,1,567.547,"POLYGON ((4.74609 53.61328, 5.09766 43.06641, ..."


## Upload

In [4]:
dataset_name = ge.upload_dataframe(gdf)

dataset_name

3c1050fe-2db9-4982-a49a-a4ab7f9ca9d2:de02beb3-f3c5-407d-a942-51d6c49ba8e0

## TODO: do something cool with the data

## Check data

In [5]:
workflow = ge.register_workflow(
    {
        "type": "Vector",
        "operator": {"type": "OgrSource", "params": {"data": str(dataset_name), "attributeProjection": None}},
    }
)

workflow.get_result_descriptor()

Data type:         MultiPolygon
Spatial Reference: EPSG:4326
Columns:
  index:
    Column Type: int
    Measurement: unitless
  label:
    Column Type: text
    Measurement: unitless
  rnd:
    Column Type: float
    Measurement: unitless

## Load data

In [6]:
time = datetime.strptime("2014-04-01T12:00:00.000Z", "%Y-%m-%dT%H:%M:%S.%f%z")

data = workflow.get_dataframe(
    ge.QueryRectangle(
        ge.BoundingBox2D(-180., -90., 180., 90.),
        ge.TimeInterval(time, time)
    )
)

data

/home/droenner/git/geoengine-python/geoengine/workflow.py:292: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data["start"] = gpd.pd.to_datetime(start, errors="coerce")
/home/droenner/git/geoengine-python/geoengine/workflow.py:293: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data["end"] = gpd.pd.to_datetime(end, errors="coerce")


,geometry,index,label,rnd,start,end
0,"POLYGON ((-121.46484 47.10938, -99.31641 17.22...",0,NA,34.340,NaT,NaT
1,"POLYGON ((4.74609 53.61328, 5.09766 43.06641, ...",1,DE,567.547,NaT,NaT
